Goal: Improve accuracy by:
1. increasing hidden layers
2. increasing epochs
3. other minor tweaks
4. also gotta add validation set
4. also understand that thing lol

In [66]:
import torch

In [67]:
import torchvision.datasets as datasets
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
])

train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

In [68]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split

train_split = 0.8
train_size = int(train_split * len(train_data))
val_size = len(train_data) - train_size

train_subset, val_subset = random_split(train_data, [train_size, val_size])

train_loader = DataLoader(dataset=train_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_subset, batch_size=64, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)


In [69]:
import torch.nn as nn
import torch.optim as optim

class Skynet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.hidden1(x)
        x = self.relu1(x)
        x = self.output(x)
        return x

In [70]:
input_size = 28 ** 2
hidden_size = 64
output_size = 10

model = Skynet(input_size, hidden_size, output_size)


In [71]:
loss_func = torch.nn.CrossEntropyLoss()

In [72]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

In [73]:
def train_one_epoch():
    model.train(True)
    batches = 0
    avg_loss = 0
    for step, (features, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        preds = model(features)
        loss = loss_func(preds, labels)
        loss.backward()
        optimizer.step()
        
        avg_loss += loss
        batches = step
    
    avg_loss = avg_loss / batches
    print(f"Average loss for training batches in this epoch: {avg_loss}")

    model.train(False)
    batches = 0
    avg_loss = 0
    for step, (features, labels) in enumerate(val_loader):
        preds = model(features)
        loss = loss_func(preds, labels)
        
        avg_loss += loss
        batches = step

    avg_loss = avg_loss / batches
    print(f"Average loss for validation batches in this epoch: {avg_loss}")


In [74]:
model.train(True)
for i in range (0, 3):
    print(f"Beginning epoch {i}...")
    train_one_epoch()
    print("")

Beginning epoch 0...
Average loss for training batches in this epoch: 0.3898734152317047
Average loss for validation batches in this epoch: 0.22706986963748932

Beginning epoch 1...
Average loss for training batches in this epoch: 0.18591928482055664
Average loss for validation batches in this epoch: 0.1651688814163208

Beginning epoch 2...
Average loss for training batches in this epoch: 0.13877534866333008
Average loss for validation batches in this epoch: 0.14361445605754852



In [75]:
model.train(False)
preds = []
for features, labels in test_loader:
    with torch.no_grad():
        batch_preds = model(features)
        preds.extend(batch_preds.tolist())

In [76]:
preds_tensor = torch.tensor(preds)
category_preds = torch.argmax(preds_tensor, dim=1)

In [77]:
def check_accuracy(preds, actual):
    if len(actual) != len(preds):
        return -1
    return sum([int(actual[i] == preds[i]) for i in range(0, len(actual))]) / len(actual)

In [78]:
print(f"Accuracy: {check_accuracy(category_preds, test_data.targets)}")

Accuracy: 0.9588


Sick, about 97.5% accuracy now! Big improvement from 85% in the last notebook. Next step: implement recurrency.